In [2]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
def dataset_value(dataset, lon, lat):
    point = ee.Geometry.Point([lon, lat])
    if isinstance(dataset, ee.imagecollection.ImageCollection):
        dataset = dataset.mean()
    value = dataset.reduceRegion(
        reducer=ee.Reducer.first(), 
        geometry=point, 
        scale=30
    ).getInfo()
    return value

In [4]:
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
temperature = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Day_1km')
biomass = ee.ImageCollection("WCMC/biomass_carbon_density/v1_0").select('carbon_tonnes_per_ha')
fires = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")

In [5]:
n=2
fire_feature = ee.Feature(fires.toList(n).get(n-1))
fire_bounds = fire_feature.geometry().bounds()
buffer_distance = 1

# Create a rectangle around the fire perimeter with a buffer
rectangle = fire_bounds.buffer(buffer_distance).bounds()

In [7]:
def generate_grid(corners, point_spacing):
    coords = []
    min_lon, min_lat = corners[0]
    max_lon, max_lat = corners[1]

    lon = min_lon
    while lon < max_lon:
        lat = min_lat
        while lat < max_lat:
            coords.append([lon, lat])
            lat += point_spacing
        lon += point_spacing
    return coords


In [12]:
# Define the spacing between points (in degrees)
point_spacing = 0.001  # Example: approximately 111 meters at the equator

# Extract the lower-left and upper-right corners of the rectangle
rect_coords = rectangle.bounds().getInfo()['coordinates'][0]
lower_left = rect_coords[0]  # Lower-left corner
upper_right = rect_coords[2]  # Upper-right corner
    
grid_points = generate_grid([lower_left, upper_right], point_spacing)
print(lower_left, upper_right)
center = [0, 0]
center[0] = (lower_left[0]+upper_right[0])/2
center[1] = (lower_left[1]+upper_right[1])/2
print(center)

[-117.57384832705529, 45.85785001398291] [-117.44666230316773, 45.941703618149845]
[-117.51025531511151, 45.89977681606638]


In [28]:
data = []
for lon, lat in grid_points:
    elev = dataset_value(elevation, lon, lat)
    temp = dataset_value(temperature, lon, lat)
    bio = dataset_value(biomass, lon, lat)
    burned = fire_feature.geometry().contains(ee.Geometry.Point([lon, lat])).getInfo()
    data.append([elev['elevation'], temp['LST_Day_1km'], bio['carbon_tonnes_per_ha'], burned])

KeyboardInterrupt: 

In [32]:
import csv

# Open a new CSV file
with open('output.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row of the array to the CSV
    for row in data:
        writer.writerow(row)

In [13]:
import csv

# Initialize an empty list to hold the 2D array
datapp = []

# Open the CSV file
with open('output.csv', newline='') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Iterate over each row in the CSV file
    for row in reader:
        # Append the row to your data list
        datapp.append(row)

# Now, 'data' is a 2D array containing your CSV file data
print(datapp)

[['870', '14590.814870635519', '18.355554580688477', 'False'], ['853', '14604.709556057185', '54.5513916015625', 'False'], ['831', '14604.709556057185', '54.5513916015625', 'False'], ['799', '14604.709556057185', '54.5513916015625', 'False'], ['774', '14604.709556057185', '57.01499938964844', 'False'], ['755', '14604.709556057185', '57.01499938964844', 'False'], ['722', '14615.632765531062', '57.01499938964844', 'False'], ['694', '14615.632765531062', '44.41539001464844', 'False'], ['693', '14615.632765531062', '44.41539001464844', 'False'], ['672', '14764.49231970841', '44.41539001464844', 'False'], ['667', '14764.49231970841', '37.37649917602539', 'False'], ['677', '14764.49231970841', '37.37649917602539', 'False'], ['676', '14696.615306639635', '37.37649917602539', 'False'], ['681', '14696.615306639635', '36.60222244262695', 'False'], ['682', '14696.615306639635', '36.60222244262695', 'False'], ['677', '14696.615306639635', '62.01260757446289', 'False'], ['693', '14696.615306639635'

In [14]:
for i in range(len(datapp)):
    lon, lat = grid_points[i]
    datapp[i] += [(lon-center[0])**2+(lat-center[1])**2]
print(datapp)

[['870', '14590.814870635519', '18.355554580688477', 'False', 0.005801927901027936], ['853', '14604.709556057185', '54.5513916015625', 'False', 0.005719074296861195], ['831', '14604.709556057185', '54.5513916015625', 'False', 0.005638220692694443], ['799', '14604.709556057185', '54.5513916015625', 'False', 0.0055593670885276825], ['774', '14604.709556057185', '57.01499938964844', 'False', 0.005482513484360912], ['755', '14604.709556057185', '57.01499938964844', 'False', 0.005407659880194132], ['722', '14615.632765531062', '57.01499938964844', 'False', 0.005334806276027344], ['694', '14615.632765531062', '44.41539001464844', 'False', 0.005263952671860546], ['693', '14615.632765531062', '44.41539001464844', 'False', 0.0051950990676937385], ['672', '14764.49231970841', '44.41539001464844', 'False', 0.005128245463526922], ['667', '14764.49231970841', '37.37649917602539', 'False', 0.005063391859360096], ['677', '14764.49231970841', '37.37649917602539', 'False', 0.00500053825519326], ['676',

In [15]:
import csv

# Open a new CSV file
with open('outputpp.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row of the array to the CSV
    for row in datapp:
        writer.writerow(row)